In [1]:
import numpy as np

In [2]:
class Lab1:
    def __init__(self, n):
        self.n = n


    def get_X(self, begin, end, size) : return np.random.normal(begin, end, size = size)


    def get_a(self, X) : return np.divide(np.sum(X), len(X))


    def get_sigma(self, X) : return np.sqrt(np.divide(np.sum((X - self.get_a(X)) ** 2), len(X)))


    def solve_A(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i); a = self.get_a(X); sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i - 1); r = a + z * sigma / np.sqrt(n_i - 1)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_B(self, z, expected):
        for n_i in self.n:
            X = self.get_X(expected, 1, n_i); a = self.get_a(X); sigma = self.get_sigma(X)
            l = a - z * sigma / np.sqrt(n_i) ;r = a + z * sigma / np.sqrt(n_i)
#             print(type(left), type(right), type(expected))
#             print(left)
            print(f'n = {n_i}: ',\
                  f'\n\t{expected} попал в интервал ({l}, {r})\n' if expected > l and expected < r \
                  else f'\n\t{expected} попал в интервал ({l}, {r})\n')
    
    
    def solve_C(self, z, variety):
        for n_i, z_i in zip(self.n, z):
            X = self.get_X(0, variety, n_i); sigma = self.get_sigma(X)
            l = n_i * sigma / z_i[1]; r = n_i * sigma / z_i[0]
            print(f'n = {n_i}: ',\
                  f'\n\t{variety} попал в интервал ({l}, {r})\n' if variety > l and variety < r \
                  else f'\n\t{variety} попал в интервал ({l}, {r})\n')

In [3]:
L1 = Lab1(np.array([100, 10000, 1000000]))

In [4]:
L1.solve_A(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.20668703492595386, 0.3000484200611664)

n = 10000:  
	0.0 попал в интервал (-0.023913851850291188, 0.02708035372317941)

n = 1000000:  
	0.0 попал в интервал (-0.002602474312651049, 0.0025496575077886678)



In [5]:
L1.solve_B(2.575, .0)

n = 100:  
	0.0 попал в интервал (-0.36040163318424306, 0.17239966055658845)

n = 10000:  
	0.0 попал в интервал (-0.03997998090886792, 0.011836038792869867)

n = 1000000:  
	0.0 попал в интервал (-0.003418846723981696, 0.0017345108620582723)



In [6]:
z = [(66.5101053, 138.9867835), (9638.498197, 10367.01484), (996359.9858, 1003645.527)]

L1.solve_C(z, 1.)

n = 100:  
	1.0 попал в интервал (0.6735186696174764, 1.4074582064649386)

n = 10000:  
	1.0 попал в интервал (0.9624993722668191, 1.0352489642926472)

n = 1000000:  
	1.0 попал в интервал (0.9973112043259859, 1.0046037009857192)



In [7]:
from numba import jit
# оформить все это в виде класса не получилось, ведь Numba плохо приспособлена для этого

In [8]:
@jit(nopython = True, parallel = True)
def get_sequence(size) : return np.random.uniform(0.0, 1.0, size = int(size))

In [9]:
@jit(nopython = True, parallel = True)
def get_ksi(w, alpha) : return ((-np.log(w)) ** 0.25) / alpha

In [10]:
@jit(nopython = True, parallel = True)
def get_etha(w) : return np.sqrt(-np.log(w))

In [11]:
@jit(nopython = True, parallel = True)
def get_count_nonzero(arr) : return np.count_nonzero(arr == 1)

In [12]:
@jit(nopython = True, parallel = True)
def task_B(size, alpha):
    k = get_ksi(get_sequence(size), alpha); t = get_etha(get_sequence(size))
    return k < t

In [13]:
@jit(nopython = True, parallel = True)
def task_C(size, alpha):
    k = get_ksi(get_sequence(size), alpha)
    return np.exp(-(k ** 2))

In [14]:
@jit(nopython = True, parallel = True)
def task_D(size, alpha):
    t = get_etha(get_sequence(size))
    return 1 - np.exp( - ((alpha * t) ** 4))

In [15]:
@jit(nopython = True, parallel = True)
def task_E(size, alpha):
    t1 = -np.log(get_sequence(size))
    t2 = -np.log(get_sequence(size))
    t3 = -np.log(get_sequence(size))
    b = np.sqrt(t1 + t2 + t3)
    return 2 * ( 1 - np.exp( -((alpha * b) ** 4) )) / (b ** 4)

In [16]:
@jit(nopython = True, parallel = True)
def calc_result(n_0, alpha, task):
    z2 = 2.575 ** 2; e2 = 0.01 ** 2; n = n_0
    sequence = task(n_0, alpha)
    summ = sequence.sum(); summ_squared = (sequence ** 2).sum()

    assert summ != 0.0, 'Нужно n_0 побольше' 
    Q = summ / n
    sigma2 = (summ_squared - n * Q * Q) / (n - 1)
    criteria = (z2 * sigma2) / (e2 * Q * Q)
    step = 10000
    while n < criteria:
        n += step
        q = task(step, alpha)
        summ += q.sum(); summ_squared += (q ** 2).sum()
        Q = summ / n
        sigma2 = (summ_squared - n * Q * Q) / (n - 1)
        criteria = (z2 * sigma2) / (e2 * Q * Q)

    while n > np.ceil(criteria) and criteria >= 0:
        n -= 1
        q = task(1, alpha)
        summ -= q.sum(); summ_squared -= (q ** 2).sum()
        Q = summ / n
        if n - 1 <= 0 or criteria <= 0:
            break
        sigma2 = (summ_squared - n * Q * Q) / (n - 1)
        criteria = (z2 * sigma2) / (e2 * Q * Q)
    return alpha, n, Q

In [20]:
labels = np.array(['alpha', 'n*', 'Q'])
tasks = np.array([task_B, task_C, task_D, task_E])
iterations = np.array([10000, 10000, 10000])
alphas = np.array([1.0, 0.3, 0.1])


# @jit(nopython = True, parallel = True)
def do_all_stuff(labels, tasks):
    for task in tasks:
        print(f'{str(task)[24 : 30]}\n')
        for it, alpha in zip(iterations, alphas):
            if 'task_B' in str(task) and alpha == 0.1:
                it = 1000000
            result = calc_result(it, alpha, task)
            for label, value in zip(labels, result):
                print(f'{label} = {value}')
            print('\n\n')

In [21]:
%%time
do_all_stuff(labels, tasks)

task_B

alpha = 1.0
n* = 79079
Q = 0.4560755699996206



alpha = 0.3
n* = 4241852
Q = 0.015390683126143959



alpha = 0.1
n* = 332799557
Q = 0.00019919798150452465



task_C

alpha = 1.0
n* = 11362
Q = 0.4527645803817915



alpha = 0.3
n* = 1041083
Q = 0.015502051920900553



alpha = 0.1
n* = 83199210
Q = 0.0001999155604700998



task_D

alpha = 1.0
n* = 45694
Q = 0.45215217684175235



alpha = 0.3
n* = 287164
Q = 0.015503112077723752



alpha = 0.1
n* = 330810
Q = 0.00019849429081032551



task_E

alpha = 1.0
n* = 62844
Q = 0.4561171667187506



alpha = 0.3
n* = 615
Q = 0.015593794881330062



alpha = 0.1
n* = 329
Q = 0.0001998873037524341



CPU times: user 7min 17s, sys: 3min 4s, total: 10min 21s
Wall time: 3min 40s
